In [1]:
!pip install formatron

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.6/290.6 kB 7.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.7 MB/s eta 0:00:0000:01


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
from formatron.schemas.pydantic import ClassSchema
from formatron.integrations.transformers import create_formatter_logits_processor_list
from formatron.formatter import FormatterBuilder
import transformers
import torch

In [15]:
from typing import Optional, List

In [17]:
class CorporateEvent(ClassSchema):
    summary: str
    year: Optional[int]

class ListOfCorporateEvents(ClassSchema):
    events: List[CorporateEvent]

In [18]:
f = FormatterBuilder()
schema = ListOfCorporateEvents
f.append_line(f"{f.json(schema, capture_name='json')}")
logits_processor = create_formatter_logits_processor_list(tokenizer, f)

In [19]:
prompt = """Extract all the events in the document below.

## document

Kogan.com was established in 2006 by Ruslan Kogan in his parents' garage.[3] He started with a website offering LCD televisions that would be assembled for him in Chinese factories.[4]

In October 2010, Ruslan Kogan announced that Kogan.com would expand to the United Kingdom.[5] The company officially entered the UK market on 15 November,[6] with a range of LED TVs and GPS units.[7][8] The expansion makes Kogan the only Australian-owned international consumer electronics brand.[9]

On 14 September 2011, Kogan.com began shipping products from the company's Hong Kong operation. [10] Bypassing wholesalers, distributors and retailers in this way enabled the company to offer products by brands including Apple, Canon, Nikon, Samsung, Motorola and more at low prices.[11][12]

On 15 March 2016, Kogan.com acquired Dick Smith Holding's online business.[13] With the physical retail stores shut down, the Dick Smith brand transitioned to an online-only consumer electronics store.[14]

On 7 July 2016, Kogan.com floated on the Australian Securities Exchange.[15]

In May 2020, Kogan.com acquired Matt Blatt, a leading Australian online furniture retailer, for $4.4 million.[16] Kogan.com acquired New Zealand-based online retailer Mighty Ape in December 2020 for $122.4 million.[17]

At December 2021, Kogan.com delivered $698 million in gross sales for the first half of FY22 (FY21 annual Gross Sales: $1.79 billion).

In December 2022, Kogan.com acquired furniture and homewares operator Brosa for $1.5 million after the company went into voluntary administration.[18]
"""

messages = [
    {"role": "system", "content": "You are a data extraction expert."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    logits_processor=logits_processor, # added with Formatron schema defined in previous cell
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [20]:
print(response)

{
"events": [
    {
        "summary": "Establishment",
        "year": 2006
    },
    {
        "summary": "Expansion to the UK",
        "year": 2010
    },
    {
        "summary": "Shipping from Hong Kong",
        "year": 2011
    },
    {
        "summary": "Acquisition of Dick Smith Holding's online business",
        "year": 2016
    },
    {
        "summary": "Initial Public Offering (IPO)",
        "year": 2016
    },
    {
        "summary": "Acquisition of Matt Blatt",
        "year": 2020
    },
    {
        "summary": "Acquisition of Mighty Ape",
        "year": 2020
    },
    {
        "summary": "Acquisition of Brosa",
        "year": 2022
    }
]
}



In [25]:
from formatron.schemas.schema import SubstringOf
from formatron.formatter import FormatterBuilder
from pydantic import Field
import typing

In [26]:
SOURCE_DOC = """Kogan.com was established in 2006 by Ruslan Kogan in his parents' garage.[3] He started with a website offering LCD televisions that would be assembled for him in Chinese factories.[4]

In October 2010, Ruslan Kogan announced that Kogan.com would expand to the United Kingdom.[5] The company officially entered the UK market on 15 November,[6] with a range of LED TVs and GPS units.[7][8] The expansion makes Kogan the only Australian-owned international consumer electronics brand.[9]

On 14 September 2011, Kogan.com began shipping products from the company's Hong Kong operation. [10] Bypassing wholesalers, distributors and retailers in this way enabled the company to offer products by brands including Apple, Canon, Nikon, Samsung, Motorola and more at low prices.[11][12]

On 15 March 2016, Kogan.com acquired Dick Smith Holding's online business.[13] With the physical retail stores shut down, the Dick Smith brand transitioned to an online-only consumer electronics store.[14]

On 7 July 2016, Kogan.com floated on the Australian Securities Exchange.[15]

In May 2020, Kogan.com acquired Matt Blatt, a leading Australian online furniture retailer, for $4.4 million.[16] Kogan.com acquired New Zealand-based online retailer Mighty Ape in December 2020 for $122.4 million.[17]

At December 2021, Kogan.com delivered $698 million in gross sales for the first half of FY22 (FY21 annual Gross Sales: $1.79 billion).

In December 2022, Kogan.com acquired furniture and homewares operator Brosa for $1.5 million after the company went into voluntary administration.[18]"""

In [27]:
class ExtractiveCorporateEvent(ClassSchema):
    summary: typing.Annotated[str, Field(..., substring_of=SOURCE_DOC), SubstringOf(SOURCE_DOC)]
    year: Optional[int]

class ExtractiveListOfCorporateEvents(ClassSchema):
    events: List[ExtractiveCorporateEvent]

In [28]:
f = FormatterBuilder()
schema = ExtractiveListOfCorporateEvents
f.append_line(f"{f.json(schema, capture_name='json')}")
logits_processor = create_formatter_logits_processor_list(tokenizer, f)

In [29]:
prompt = f"""Extract all the events in the document below.

## document

{SOURCE_DOC}
"""

messages = [
    {"role": "system", "content": "You are a data extraction expert."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    logits_processor=logits_processor, # added with Formatron schema defined in previous cell
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [30]:
print(response)

{
"events": [
    {
        "summary": "Kogan.com was established",
        "year": 2006
    },
    {
        "summary": "Kogan.com began shipping products from the company's Hong Kong operation",
        "year": 2011
    },
    {
        "summary": "Kogan.com floated on the Australian Securities Exchange",
        "year": 2016
    },
    {
        "summary": "Kogan.com acquired Matt Blatt, a leading Australian online furniture retailer",
        "year": 2020
    },
    {
        "summary": "Kogan.com acquired New Zealand-based online retailer Mighty Ape",
        "year": 2020
    },
    {
        "summary": "Kogan.com acquired furniture and homewares operator Brosa",
        "year": 2022
    }
]
}



# Literals

In [31]:
from typing import Literal

In [32]:
SOURCE_DOC_2 = """Yeah so I'm going with Bob Smith next week to work at Apple. You can contact me on 38-0441-1455, or drop me a message on fjupio@xmx.com.

OK thanks, Max will be there as well but he's at KMB Construction these days so maybe he'll be late
"""

In [34]:
ALLOWED_ENTITIES = ["person", "organization", "telephone number", "email address"]

In [36]:
class ExtractiveEntity(ClassSchema):
    entity: typing.Annotated[str, Field(..., substring_of=SOURCE_DOC_2), SubstringOf(SOURCE_DOC_2)]
    label: Literal[tuple(ALLOWED_ENTITIES)]

class ExtractiveListOfEntities(ClassSchema):
    events: List[ExtractiveEntity]

In [37]:
f = FormatterBuilder()
schema = ExtractiveListOfEntities
f.append_line(f"{f.json(schema, capture_name='json')}")
logits_processor = create_formatter_logits_processor_list(tokenizer, f)

In [38]:
prompt = f"""Extract all the personal data in the document below.

## document

{SOURCE_DOC_2}
"""

messages = [
    {"role": "system", "content": "You are a personal data extraction expert."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    logits_processor=logits_processor, # added with Formatron schema defined in previous cell
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [40]:
print(response)

{
  "events": [
    {
      "entity": "Bob Smith",
      "label": "person"
    },
    {
      "entity": "Apple",
      "label": "organization"
    },
    {
      "entity": "38-0441-1455",
      "label": "telephone number"
    },
    {
      "entity": "fjupio@xmx.com",
      "label": "email address"
    },
    {
      "entity": "Max",
      "label": "person"
    },
    {
      "entity": "KMB Construction",
      "label": "organization"
    }
  ]
}

